In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import re
import calendar
from operator import attrgetter
import datetime

## Read in All Data

In [45]:
source_dir = '/mnt/c/Users/natra/Documents/Research/Iraq-post-conflict-rebel-governance'
iom_adm3_mapping = source_dir + '/data/iom/iom_adm3_mapping.csv'
iom_data_file = source_dir + '/data/iom/iom_raw.xlsx'
ag_indices_mask_file = source_dir + '/data/satellite_indices/evi/evi_mask_mean_max_min/evi_stats_Jan2001-Dec2022.csv'
ag_indices_nomask_file = source_dir + '/data/satellite_indices/evi/evi_mean_max_nomask/evi_stats_Feb2000-Feb2024.csv'

ndvi_indices_file = source_dir + '/data/satellite_indices/ndvi/ndvi_mask_mean_min_max/ndvi_stats_Jan2001-Dec2022.csv'

adm3_xsub_file = source_dir + '/data/conflict/adm3_xsub_monthly.csv'
pop_dir = source_dir + '/data/population/'
adm3_sunni_eth_file = source_dir + '/data/EPR/adm3_sunni_epr.csv'
territorial_control_file = source_dir + '/data/territorial-control/isis territorial control.xlsx'
ucdp_conflict_file = source_dir + '/data/conflict/ucdp_conflict_monthly.csv'
liveuamap_territorial_control_file = source_dir + '/data/liveuamap/adm3_liveuamap_isis_perc_control_monthly.csv'


# NOTE - as-is, there is a duplicate ADM3_PCODE in this shapefile!! 
# IQG01Q05N01 is for both Abi Gharaq and Markaz Ana
iraq_shp_adm3_loc = source_dir + '/data/geo_boundaries/irq-administrative-divisions-shapefiles/irq_admbnda_adm3_cso_20190603.shp'

In [46]:
iom_adm3_map = pd.read_csv(iom_adm3_mapping)
iom_data = pd.read_excel(iom_data_file, sheet_name="Full Dataset",header=1)
ag_mask_data = pd.read_csv(ag_indices_mask_file)
ag_nomask_data = pd.read_csv(ag_indices_nomask_file)

ndvi_data = pd.read_csv(ndvi_indices_file)

adm3_xsub = pd.read_csv(adm3_xsub_file)
iraq_shp_adm3 = gpd.read_file(iraq_shp_adm3_loc)
adm3_sunni_eth = pd.read_csv(adm3_sunni_eth_file)
iom_territorial_control = pd.read_excel(territorial_control_file)
liveuamap_territorial_control = pd.read_csv(liveuamap_territorial_control_file)

ucdp_conflict = pd.read_csv(ucdp_conflict_file)

In [47]:
print(iom_territorial_control.shape)
iom_territorial_control = iom_territorial_control.drop(columns=['adm2_en','adm1_en'])
iom_terr_contrl_occ = iom_territorial_control[iom_territorial_control['iom_occupied']==1]
iom_terr_contrl_occ.loc[:,'iom_occ_start_month_st'] = pd.to_datetime(iom_terr_contrl_occ[['start_yr','start_m']].\
                                    astype('Int64').astype(str).apply(" ".join,1),format="%Y %m")
iom_terr_contrl_occ.loc[:,'iom_occ_end_month_st'] = pd.to_datetime(iom_terr_contrl_occ[['end_yr','end_m']].\
                                    astype('Int64').astype(str).apply(" ".join,1),format="%Y %m")

iom_terr_contrl_occ.loc[:,'iom_occ_start_month'] = (iom_terr_contrl_occ['iom_occ_start_month_st'] + pd.offsets.MonthEnd()).dt.strftime('%Y-%m-%d')
iom_terr_contrl_occ.loc[:,'iom_occ_end_month'] = (iom_terr_contrl_occ['iom_occ_end_month_st'] + pd.offsets.MonthEnd()).dt.strftime('%Y-%m-%d')
iom_terr_contrl_occ.loc[:,'iom_occ_length_mon'] = (iom_terr_contrl_occ['iom_occ_end_month_st'].dt.to_period('M')  - iom_terr_contrl_occ['iom_occ_start_month_st'].dt.to_period('M')).apply(attrgetter('n')).astype('Int64')

iom_terr_control_full = pd.concat([iom_terr_contrl_occ,iom_territorial_control[iom_territorial_control['iom_occupied']!=1]])
iom_terr_control_full = iom_terr_control_full.loc[:,('adm3_en','iom_occupied','iom_occ_start_month','iom_occ_end_month','iom_occ_length_mon')]
print(iom_terr_control_full.shape)

(81, 8)
(81, 5)


/tmp/ipykernel_2200/850614612.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iom_terr_contrl_occ.loc[:,'iom_occ_start_month_st'] = pd.to_datetime(iom_terr_contrl_occ[['start_yr','start_m']].\
/tmp/ipykernel_2200/850614612.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iom_terr_contrl_occ.loc[:,'iom_occ_end_month_st'] = pd.to_datetime(iom_terr_contrl_occ[['end_yr','end_m']].\
/tmp/ipykernel_2200/850614612.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [48]:
# obtain desired info from IOM
iom_data_filt = iom_data.loc[:,('LocationID','DateofInterview','Q1.10 IsLocationInDisputedArea',
                                'Q1.11 WasLocationOccupied','Q1.11.1 RetakenPeriod',
                                'Q1.12 FirstMajorEthno-religiousPrior','Q1.13 FirstMajorEthno-religiousCurrent')]

In [49]:
# concatenate population year files
# processing of global tif files into iraq adm3 regions done in Google Colab Pro - 
# below I'm reading in the output of the Colab files

pop_dfs = []
for file in os.listdir(pop_dir):
    if file.endswith('.csv'):
        file_path = pop_dir + file
        df = pd.read_csv(file_path, index_col=None)
        pop_dfs.append(df)
pop_df_full = pd.concat(pop_dfs, axis=0, ignore_index=True)

## Merge all data at monthly, adm3 level

### Agricultural Indices - Masked for Cropland

In [50]:
# get max evi at monthly level
ag_mask_data_max = ag_mask_data.loc[:,('ADM3_EN','max_evi','max_evi_scaled','date')]
ag_mask_data_max['date'] = pd.to_datetime(ag_mask_data_max['date'])
ag_mask_data_max = ag_mask_data_max.set_index('date')
ag_mask_data_max_month = ag_mask_data_max.groupby(['ADM3_EN'])[['max_evi','max_evi_scaled']].resample('ME').max()
ag_mask_data_max_month = ag_mask_data_max_month.rename(columns={'max_evi':'max_evi_mask','max_evi_scaled':'max_evi_mask_scaled'})

ag_mask_data_max_month.head()

max_evi_mask  max_evi_mask_scaled
ADM3_EN    date                                         
Abi Gharaq 2001-01-31        5416.0               0.5416
           2001-02-28        6291.0               0.6291
           2001-03-31        6513.0               0.6513
           2001-04-30        5332.0               0.5332
           2001-05-31        5147.0               0.5147

In [51]:
# get mean evi at monthly level
ag_mask_data_avg = ag_mask_data.loc[:,('ADM3_EN','mean_evi','mean_evi_scaled','date')]
ag_mask_data_avg['date'] = pd.to_datetime(ag_mask_data_avg['date'])
ag_mask_data_avg = ag_mask_data_avg.set_index('date')
ag_mask_data_avg_month = ag_mask_data_avg.groupby(['ADM3_EN'])[['mean_evi','mean_evi_scaled']].resample('ME').mean()
ag_mask_data_avg_month = ag_mask_data_avg_month.rename(columns={'mean_evi':'mean_evi_mask','mean_evi_scaled':'mean_evi_mask_scaled'})

ag_mask_data_avg_month.head()

mean_evi_mask  mean_evi_mask_scaled
ADM3_EN    date                                           
Abi Gharaq 2001-01-31    2886.373490              0.288637
           2001-02-28    3094.206464              0.309421
           2001-03-31    3275.809481              0.327581
           2001-04-30    2596.503366              0.259650
           2001-05-31    2161.217474              0.216122

In [52]:
ag_mask_data_monthly = ag_mask_data_avg_month.join(ag_mask_data_max_month)
print(ag_mask_data_avg_month.shape)
print(ag_mask_data_max_month.shape)
print(ag_mask_data_monthly.shape)

(77616, 2)
(77616, 2)
(77616, 4)


In [53]:
# get max NDVI at monthly level
ndvi_data_max = ndvi_data.loc[:,('ADM3_EN','max_ndvi','max_ndvi_scaled','date')]
ndvi_data_max['date'] = pd.to_datetime(ndvi_data_max['date'])
ndvi_data_max = ndvi_data_max.set_index('date')
ndvi_data_max_month = ndvi_data_max.groupby(['ADM3_EN'])[['max_ndvi','max_ndvi_scaled']].resample('ME').max()

ndvi_data_max_month.head()

max_ndvi  max_ndvi_scaled
ADM3_EN    date                                 
Abi Gharaq 2001-01-31    7444.0           0.7444
           2001-02-28    7638.0           0.7638
           2001-03-31    8156.0           0.8156
           2001-04-30    7049.0           0.7049
           2001-05-31    5842.0           0.5842

In [54]:
# get mean ndvi at monthly level
ndvi_data_avg = ndvi_data.loc[:,('ADM3_EN','mean_ndvi','mean_ndvi_scaled','date')]
ndvi_data_avg['date'] = pd.to_datetime(ndvi_data_avg['date'])
ndvi_data_avg = ndvi_data_avg.set_index('date')
ndvi_data_avg_month = ndvi_data_avg.groupby(['ADM3_EN'])[['mean_ndvi','mean_ndvi_scaled']].resample('ME').mean()

ndvi_data_avg_month.head()

mean_ndvi  mean_ndvi_scaled
ADM3_EN    date                                     
Abi Gharaq 2001-01-31  4758.344221          0.475834
           2001-02-28  4635.300171          0.463530
           2001-03-31  4878.448034          0.487845
           2001-04-30  3802.582658          0.380258
           2001-05-31  2919.671284          0.291967

In [55]:
ndvi_data_monthly = ndvi_data_avg_month.join(ndvi_data_max_month)
print(ndvi_data_avg_month.shape)
print(ndvi_data_max_month.shape)
print(ndvi_data_monthly.shape)

(77616, 2)
(77616, 2)
(77616, 4)


### Agricultural Indicies - No Mask

In [56]:
# get max evi at monthly level
ag_nomask_data_max = ag_nomask_data.loc[:,('ADM3_EN','max_evi','max_evi_scaled','date')]
ag_nomask_data_max['date'] = pd.to_datetime(ag_nomask_data_max['date'])
ag_nomask_data_max = ag_nomask_data_max.set_index('date')
ag_nomask_data_max_month = ag_nomask_data_max.groupby(['ADM3_EN'])[['max_evi','max_evi_scaled']].resample('ME').max()
ag_nomask_data_max_month = ag_nomask_data_max_month.rename(columns={'max_evi':'max_evi_nomask','max_evi_scaled':'max_evi_nomask_scaled'})

ag_nomask_data_max_month.head()

max_evi_nomask  max_evi_nomask_scaled
ADM3_EN    date                                             
Abi Gharaq 2000-02-29            4324                 0.4324
           2000-03-31            5047                 0.5047
           2000-04-30            4052                 0.4052
           2000-05-31            4172                 0.4172
           2000-06-30            3800                 0.3800

In [57]:
# get mean evi at monthly level
ag_nomask_data_avg = ag_nomask_data.loc[:,('ADM3_EN','mean_evi','mean_evi_scaled','date')]
ag_nomask_data_avg['date'] = pd.to_datetime(ag_nomask_data_avg['date'])
ag_nomask_data_avg = ag_nomask_data_avg.set_index('date')
ag_nomask_data_avg_month = ag_nomask_data_avg.groupby(['ADM3_EN'])[['mean_evi','mean_evi_scaled']].resample('ME').mean()
ag_nomask_data_avg_month = ag_nomask_data_avg_month.rename(columns={'mean_evi':'mean_evi_nomask','mean_evi_scaled':'mean_evi_nomask_scaled'})


ag_nomask_data_avg_month.head()

mean_evi_nomask  mean_evi_nomask_scaled
ADM3_EN    date                                               
Abi Gharaq 2000-02-29      2078.441277                0.207844
           2000-03-31      2269.792899                0.226979
           2000-04-30      2080.297912                0.208030
           2000-05-31      1915.994029                0.191599
           2000-06-30      1782.604809                0.178260

In [58]:
ag_nomask_data_monthly = ag_nomask_data_avg_month.join(ag_nomask_data_max_month)
print(ag_nomask_data_avg_month.shape)
print(ag_nomask_data_max_month.shape)
print(ag_nomask_data_monthly.shape)

(84966, 2)
(84966, 2)
(84966, 4)


### X-Sub Conflict Data

In [59]:
# xsub only goes until 2019-12-31
adm3_xsub_mon = adm3_xsub.rename(columns={'month':'date'})
adm3_xsub_mon = adm3_xsub_mon.drop(columns=['ADM3_PCODE'])
adm3_xsub_mon = adm3_xsub_mon.set_index(['ADM3_EN','date'])
adm3_xsub_mon.head()

ACTION_IND  ACTION_DIR  ACTION_PRT
ADM3_EN    date                                          
Abi Gharaq 2017-12-31           1           1           0
           2018-01-31           0           0           0
           2018-02-28           0           0           0
           2018-03-31           0           0           0
           2018-04-30           0           0           0

### Combine Ag Indices and X-Sub Conflict

In [60]:
# join agricultural EVI indices with xsub conflict data
adm3_iraq_data = ag_nomask_data_monthly.join(adm3_xsub_mon,how='outer')
adm3_iraq_data = adm3_iraq_data.join(ag_mask_data_monthly,how='outer')
adm3_iraq_data = adm3_iraq_data.join(ndvi_data_monthly,how='outer')

print(ag_nomask_data_monthly.shape)
print(ag_mask_data_monthly.shape)
print(ndvi_data_monthly.shape)
print(adm3_xsub_mon.shape)
print(adm3_iraq_data.shape)
adm3_iraq_data = adm3_iraq_data.reset_index()

(84966, 4)
(77616, 4)
(77616, 4)
(23658, 3)
(84966, 15)


In [61]:
adm3_iraq_data.head()

,ADM3_EN,date,mean_evi_nomask,mean_evi_nomask_scaled,max_evi_nomask,max_evi_nomask_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT,mean_evi_mask,mean_evi_mask_scaled,max_evi_mask,max_evi_mask_scaled,mean_ndvi,mean_ndvi_scaled,max_ndvi,max_ndvi_scaled
0,Abi Gharaq,2000-02-29,2078.441277,0.207844,4324,0.4324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Abi Gharaq,2000-03-31,2269.792899,0.226979,5047,0.5047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Abi Gharaq,2000-04-30,2080.297912,0.208030,4052,0.4052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Abi Gharaq,2000-05-31,1915.994029,0.191599,4172,0.4172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Abi Gharaq,2000-06-30,1782.604809,0.178260,3800,0.3800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
# no duplicates at this point
adm3_iraq_data[adm3_iraq_data.duplicated(subset=['date','ADM3_EN'],keep=False)]

,ADM3_EN,date,mean_evi_nomask,mean_evi_nomask_scaled,max_evi_nomask,max_evi_nomask_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT,mean_evi_mask,mean_evi_mask_scaled,max_evi_mask,max_evi_mask_scaled,mean_ndvi,mean_ndvi_scaled,max_ndvi,max_ndvi_scaled


In [63]:
# make xsub conflict values 0 for un-defined time periods where we have xsub data; otherwise 
# keep NA. Have data from 2001-01-01 until 2019-12-31
min_xsub = pd.to_datetime('2001-01-01')
max_xsub = adm3_xsub.month.max()

adm3_iraq_data['ACTION_IND'] = np.where(((adm3_iraq_data['date']>=min_xsub)&(adm3_iraq_data['date']<=max_xsub)&(adm3_iraq_data.ACTION_IND.isna())),0,adm3_iraq_data.ACTION_IND)
adm3_iraq_data['ACTION_DIR'] = np.where(((adm3_iraq_data['date']>=min_xsub)&(adm3_iraq_data['date']<=max_xsub)&(adm3_iraq_data.ACTION_DIR.isna())),0,adm3_iraq_data.ACTION_DIR)
adm3_iraq_data['ACTION_PRT'] = np.where(((adm3_iraq_data['date']>=min_xsub)&(adm3_iraq_data['date']<=max_xsub)&(adm3_iraq_data.ACTION_PRT.isna())),0,adm3_iraq_data.ACTION_PRT)
adm3_iraq_data.head()

,ADM3_EN,date,mean_evi_nomask,mean_evi_nomask_scaled,max_evi_nomask,max_evi_nomask_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT,mean_evi_mask,mean_evi_mask_scaled,max_evi_mask,max_evi_mask_scaled,mean_ndvi,mean_ndvi_scaled,max_ndvi,max_ndvi_scaled
0,Abi Gharaq,2000-02-29,2078.441277,0.207844,4324,0.4324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Abi Gharaq,2000-03-31,2269.792899,0.226979,5047,0.5047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Abi Gharaq,2000-04-30,2080.297912,0.208030,4052,0.4052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Abi Gharaq,2000-05-31,1915.994029,0.191599,4172,0.4172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Abi Gharaq,2000-06-30,1782.604809,0.178260,3800,0.3800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# no duplicates at this point
adm3_iraq_data[adm3_iraq_data.duplicated(subset=['date','ADM3_EN'],keep=False)]

,ADM3_EN,date,mean_evi_nomask,mean_evi_nomask_scaled,max_evi_nomask,max_evi_nomask_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT,mean_evi_mask,mean_evi_mask_scaled,max_evi_mask,max_evi_mask_scaled,mean_ndvi,mean_ndvi_scaled,max_ndvi,max_ndvi_scaled


In [65]:
# create 5-year interval fields to join with population data
adm3_iraq_data['year'] = adm3_iraq_data['date'].dt.year
adm3_iraq_data['year_5yr'] = np.where(
                               adm3_iraq_data['year'] < 2005, 2000,
                               np.where(adm3_iraq_data['year'] < 2010, 2005,
                                np.where(adm3_iraq_data['year'] < 2015, 2010,
                                np.where(adm3_iraq_data['year'] < 2020, 2015,
                                np.where(adm3_iraq_data['year'] < 2025, 2020, 0)))))
adm3_iraq_data.head()

,ADM3_EN,date,mean_evi_nomask,mean_evi_nomask_scaled,max_evi_nomask,max_evi_nomask_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT,mean_evi_mask,mean_evi_mask_scaled,max_evi_mask,max_evi_mask_scaled,mean_ndvi,mean_ndvi_scaled,max_ndvi,max_ndvi_scaled,year,year_5yr
0,Abi Gharaq,2000-02-29,2078.441277,0.207844,4324,0.4324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,2000
1,Abi Gharaq,2000-03-31,2269.792899,0.226979,5047,0.5047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,2000
2,Abi Gharaq,2000-04-30,2080.297912,0.208030,4052,0.4052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,2000
3,Abi Gharaq,2000-05-31,1915.994029,0.191599,4172,0.4172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,2000
4,Abi Gharaq,2000-06-30,1782.604809,0.178260,3800,0.3800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,2000


In [66]:
# no duplicates at this point
adm3_iraq_data[adm3_iraq_data.duplicated(subset=['date','ADM3_EN'],keep=False)]

,ADM3_EN,date,mean_evi_nomask,mean_evi_nomask_scaled,max_evi_nomask,max_evi_nomask_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT,mean_evi_mask,mean_evi_mask_scaled,max_evi_mask,max_evi_mask_scaled,mean_ndvi,mean_ndvi_scaled,max_ndvi,max_ndvi_scaled,year,year_5yr


In [67]:
# join population data by ADM3_PCODE and 5-year interval
pop_df_join = pop_df_full.drop(columns=['Shape_Area'])
print(adm3_iraq_data.shape)
adm3_iraq_data = adm3_iraq_data.merge(pop_df_join, how='outer',left_on=['ADM3_EN','year_5yr'],
                                      right_on=['ADM3_EN','year'])
adm3_iraq_data = adm3_iraq_data.drop(columns=['year_x','year_y','year_5yr'])
print(pop_df_join.shape)
print(adm3_iraq_data.shape)

(84966, 19)
(1470, 4)
(84966, 19)


In [68]:
# no duplicates at this point
adm3_iraq_data[adm3_iraq_data.duplicated(subset=['date','ADM3_EN'],keep=False)]

,ADM3_EN,date,mean_evi_nomask,mean_evi_nomask_scaled,max_evi_nomask,max_evi_nomask_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT,mean_evi_mask,mean_evi_mask_scaled,max_evi_mask,max_evi_mask_scaled,mean_ndvi,mean_ndvi_scaled,max_ndvi,max_ndvi_scaled,pop_count,pop_density


In [69]:
adm3_iraq_data.head()

,ADM3_EN,date,mean_evi_nomask,mean_evi_nomask_scaled,max_evi_nomask,max_evi_nomask_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT,mean_evi_mask,mean_evi_mask_scaled,max_evi_mask,max_evi_mask_scaled,mean_ndvi,mean_ndvi_scaled,max_ndvi,max_ndvi_scaled,pop_count,pop_density
0,Abi Gharaq,2000-02-29,2078.441277,0.207844,4324,0.4324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137199.66,8.709119e+06
1,Abi Gharaq,2000-03-31,2269.792899,0.226979,5047,0.5047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137199.66,8.709119e+06
2,Abi Gharaq,2000-04-30,2080.297912,0.208030,4052,0.4052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137199.66,8.709119e+06
3,Abi Gharaq,2000-05-31,1915.994029,0.191599,4172,0.4172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137199.66,8.709119e+06
4,Abi Gharaq,2000-06-30,1782.604809,0.178260,3800,0.3800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137199.66,8.709119e+06


#### IOM Territorial Control

In [70]:
# format IOM data for joining
iom_data_filt = iom_data_filt.rename(columns={
                                    'Q1.10 IsLocationInDisputedArea':'disputed_area',
                                    'Q1.11 WasLocationOccupied':'loc_occupied',
                                    'Q1.11.1 RetakenPeriod':'retaken_period',
                                    'Q1.12 FirstMajorEthno-religiousPrior':'maj_ethrelig_prior',
                                    'Q1.13 FirstMajorEthno-religiousCurrent':'maj_ethrelig_curr'         
                                    })

In [71]:
# add flags for whether attacked, occupied, or no ISIL activity
iom_data_filt['iom_attacked'] = np.where(iom_data_filt['loc_occupied']=='Yes, attacked', 1, 0)
iom_data_filt['iom_occupied'] = np.where(iom_data_filt['loc_occupied']=='Yes, occupied', 1, 0)
iom_data_filt['iom_no_isil_action'] = np.where(iom_data_filt['loc_occupied']=='No', 1, 0)
# add flags for whether in ISIL religious-ethnic group prior to crisis and currently
iom_data_filt['iom_isil_ingroup_prior'] = np.where(iom_data_filt['maj_ethrelig_prior']=='Arab Sunni Muslim', 1, 0)
iom_data_filt['iom_isil_ingroup_curr'] = np.where(iom_data_filt['maj_ethrelig_curr']=='Arab Sunni Muslim', 1, 0)
# convert disputed area answers to 1,0 flags
iom_data_filt['iom_disputed_area'] = np.where(iom_data_filt['disputed_area']=='Yes', 1, 0)

In [72]:
# add flag for whether retaken pre- or post-2016 as indicator for long or short-term occupation
iom_data_filt['iom_retaken_year'] = iom_data_filt['retaken_period'].str.extract("(\d\d\d\d)")
iom_data_filt['iom_retaken_year'] = iom_data_filt['iom_retaken_year'].astype('float')
iom_data_filt['iom_retaken_post_2016'] = np.where(iom_data_filt['iom_retaken_year'].isna(),iom_data_filt['iom_retaken_year'],
                                              np.where(iom_data_filt['iom_retaken_year']>=2017,1,0))

In [73]:
# drop unnecessary columns
iom_data_filt = iom_data_filt.drop(columns=['loc_occupied','retaken_period','maj_ethrelig_prior',
                                            'maj_ethrelig_curr','DateofInterview','disputed_area'])

In [74]:
iom_adm3_data = iom_data_filt.merge(iom_adm3_map.loc[:,('LocationID','ADM3_EN')],how='left',
                                    on='LocationID')
print(iom_data_filt.shape)
print(iom_adm3_data.shape)

(3717, 9)
(3717, 10)


In [75]:
# to aggregate, want to take the max of all fields except for iom_no_isil_action, which we want to take the
# min of - if any part of the adm3 area had isil action, we don't want to flag it as not having any action
iom_adm3_data_for_min = iom_adm3_data.loc[:,('ADM3_EN','iom_no_isil_action')]
iom_adm3_data_for_max = iom_adm3_data.drop(columns=['LocationID','iom_no_isil_action'])

In [76]:
iom_adm3_data_max_gb = iom_adm3_data_for_max.groupby(['ADM3_EN']).max().reset_index()
iom_adm3_data_min_gb = iom_adm3_data_for_min.groupby(['ADM3_EN']).min().reset_index()

In [77]:
iom_adm3_data_fin = iom_adm3_data_max_gb.merge(iom_adm3_data_min_gb,how='outer',
                                               on='ADM3_EN')
print(iom_adm3_data_max_gb.shape)
print(iom_adm3_data_min_gb.shape)
print(iom_adm3_data_fin.shape)

(215, 8)
(215, 2)
(215, 9)


In [78]:
# add IOM data to full dataset
print(adm3_iraq_data.shape)
adm3_iraq_data = adm3_iraq_data.merge(iom_adm3_data_fin, how='outer',
                                      on='ADM3_EN')
print(adm3_iraq_data.shape)

(84966, 19)
(84966, 27)


In [79]:
# identify number of Adm3 areas with no IOM data
len(adm3_iraq_data[adm3_iraq_data['iom_occupied'].isna()].ADM3_EN.unique())

79

In [80]:
adm3_iraq_data[adm3_iraq_data.duplicated(subset=['ADM3_EN','date'],keep=False)]

,ADM3_EN,date,mean_evi_nomask,mean_evi_nomask_scaled,max_evi_nomask,max_evi_nomask_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT,mean_evi_mask,...,pop_count,pop_density,iom_attacked,iom_occupied,iom_isil_ingroup_prior,iom_isil_ingroup_curr,iom_disputed_area,iom_retaken_year,iom_retaken_post_2016,iom_no_isil_action


#### Liveuamap Territorial Control

In [81]:
liveuamap_territorial_control_join = liveuamap_territorial_control.drop(columns=['control_area','full_adm3_area','control_agent'])
# subtract one day from the first day of each month to get last day of prior month,
# which enables join with the original data
liveuamap_territorial_control_join.loc[:,'date_month_end'] = pd.to_datetime(liveuamap_territorial_control_join['date_dt']) - datetime.timedelta(days=1)
liveuamap_territorial_control_join = liveuamap_territorial_control_join.drop(columns=['date_dt'])

In [82]:
# add in Liveuamap Territorial Control data
print(adm3_iraq_data.shape)
adm3_iraq_data = adm3_iraq_data.merge(liveuamap_territorial_control_join, how='outer',
                                      left_on=['ADM3_EN','date'], right_on=['ADM3_EN','date_month_end'])
print(adm3_iraq_data.shape)


(84966, 27)
(84966, 29)


### GeoEPR

In [83]:
# add in GeoEPR data
adm3_sunni_eth = adm3_sunni_eth.drop(columns=['ADM3_PCODE']) 
adm3_iraq_data = adm3_iraq_data.merge(adm3_sunni_eth, on=['ADM3_EN'],how='left')
print(adm3_iraq_data.shape)
adm3_iraq_data[adm3_iraq_data.duplicated(subset=['ADM3_EN','date'],keep=False)]

(84966, 32)


,ADM3_EN,date,mean_evi_nomask,mean_evi_nomask_scaled,max_evi_nomask,max_evi_nomask_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT,mean_evi_mask,...,iom_isil_ingroup_curr,iom_disputed_area,iom_retaken_year,iom_retaken_post_2016,iom_no_isil_action,perc_isis_control,date_month_end,sunni_dom,sunni_mix,no_sunni


In [84]:

adm3_iraq_data.head(2)

,ADM3_EN,date,mean_evi_nomask,mean_evi_nomask_scaled,max_evi_nomask,max_evi_nomask_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT,mean_evi_mask,...,iom_isil_ingroup_curr,iom_disputed_area,iom_retaken_year,iom_retaken_post_2016,iom_no_isil_action,perc_isis_control,date_month_end,sunni_dom,sunni_mix,no_sunni
0,Abi Gharaq,2000-02-29,2078.441277,0.207844,4324,0.4324,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,NaN,1.0,NaN,NaT,0,0,1
1,Abi Gharaq,2000-03-31,2269.792899,0.226979,5047,0.5047,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,NaN,1.0,NaN,NaT,0,0,1


In [85]:
# attach more precise occupation dates
print(adm3_iraq_data.shape)
adm3_iraq_data = adm3_iraq_data.merge(iom_terr_control_full.drop(columns=['iom_occupied']),
                                           left_on='ADM3_EN',right_on='adm3_en',how='left')
print(adm3_iraq_data.shape)

(84966, 32)
(84966, 36)


In [86]:
adm3_iraq_data.head()

,ADM3_EN,date,mean_evi_nomask,mean_evi_nomask_scaled,max_evi_nomask,max_evi_nomask_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT,mean_evi_mask,...,iom_no_isil_action,perc_isis_control,date_month_end,sunni_dom,sunni_mix,no_sunni,adm3_en,iom_occ_start_month,iom_occ_end_month,iom_occ_length_mon
0,Abi Gharaq,2000-02-29,2078.441277,0.207844,4324,0.4324,NaN,NaN,NaN,NaN,...,1.0,NaN,NaT,0,0,1,NaN,NaN,NaN,<NA>
1,Abi Gharaq,2000-03-31,2269.792899,0.226979,5047,0.5047,NaN,NaN,NaN,NaN,...,1.0,NaN,NaT,0,0,1,NaN,NaN,NaN,<NA>
2,Abi Gharaq,2000-04-30,2080.297912,0.208030,4052,0.4052,NaN,NaN,NaN,NaN,...,1.0,NaN,NaT,0,0,1,NaN,NaN,NaN,<NA>
3,Abi Gharaq,2000-05-31,1915.994029,0.191599,4172,0.4172,NaN,NaN,NaN,NaN,...,1.0,NaN,NaT,0,0,1,NaN,NaN,NaN,<NA>
4,Abi Gharaq,2000-06-30,1782.604809,0.178260,3800,0.3800,NaN,NaN,NaN,NaN,...,1.0,NaN,NaT,0,0,1,NaN,NaN,NaN,<NA>


In [87]:
ucdp_conflict[(ucdp_conflict['ADM3_EN']=='Abu Dalf')&(ucdp_conflict['month']<='2012-06-30')]

,ADM3_EN,where_prec,isis_conflict,civ_violence,month,num_events
5,Abu Dalf,1,1,0,2008-09-30,1
10,Abu Dalf,2,1,0,2008-02-29,1
11,Abu Dalf,2,1,0,2011-02-28,1
13,Abu Dalf,4,1,0,2006-05-31,1
14,Abu Dalf,4,1,0,2007-07-31,1
15,Abu Dalf,4,1,0,2007-10-31,1
16,Abu Dalf,4,1,0,2008-01-31,3
17,Abu Dalf,4,1,0,2008-07-31,1
18,Abu Dalf,4,1,0,2008-09-30,1
19,Abu Dalf,4,1,0,2009-11-30,1


In [88]:
print(ucdp_conflict.columns)
ucdp_conflict.head()

Index(['ADM3_EN', 'where_prec', 'isis_conflict', 'civ_violence', 'month',
       'num_events'],
      dtype='object')


,ADM3_EN,where_prec,isis_conflict,civ_violence,month,num_events
0,Abi Gharaq,4,1,0,2013-06-30,1
1,Abi Gharaq,4,1,0,2013-11-30,1
2,Abi Gharaq,4,1,0,2014-06-30,1
3,Abi Gharaq,4,1,0,2014-07-31,1
4,Abi Gharaq,4,1,1,2013-06-30,1


In [95]:
# sum events at the admin 3 level of precision or better (where_prec == 1 or 2)
print(ucdp_conflict.shape)
ucdp_conflict_precadm3 = ucdp_conflict[ucdp_conflict['where_prec'].isin([1,2])]
ucdp_conflict_precadm3.loc[:,'isis_civ'] = ucdp_conflict_precadm3['isis_conflict'].astype(str) + "_" + ucdp_conflict_precadm3['civ_violence'].astype(str)
print(ucdp_conflict_precadm3.shape)
ucdp_conflict_precadm3 = ucdp_conflict_precadm3.groupby(['ADM3_EN','isis_civ','month'])['num_events'].sum().reset_index()
ucdp_conflict_precadm3 = ucdp_conflict_precadm3.pivot(index=['ADM3_EN','month'],columns='isis_civ',values='num_events').reset_index()
ucdp_conflict_precadm3 = ucdp_conflict_precadm3.rename(columns={"0_0": 'ucdp_nonisis_nonciv_events_adm3prec',
                                                                "0_1": 'ucdp_nonisis_civ_events_adm3prec',
                                                                "1_0": 'ucdp_isis_nonciv_events_adm3prec',
                                                                "1_1": 'ucdp_isis_civ_events_adm3prec'})
ucdp_conflict_precadm3 = ucdp_conflict_precadm3.fillna(value={'ucdp_nonisis_nonciv_events_adm3prec':0,
                                                               'ucdp_nonisis_civ_events_adm3prec':0,
                                                               'ucdp_isis_nonciv_events_adm3prec':0,
                                                               'ucdp_isis_civ_events_adm3prec':0})
ucdp_conflict_precadm3.loc[:,'ucdp_all_isis_events_adm3prec'] = ucdp_conflict_precadm3['ucdp_isis_nonciv_events_adm3prec'] + ucdp_conflict_precadm3['ucdp_isis_civ_events_adm3prec']
ucdp_conflict_precadm3.loc[:,'ucdp_all_civ_events_adm3prec'] = ucdp_conflict_precadm3['ucdp_nonisis_civ_events_adm3prec'] + ucdp_conflict_precadm3['ucdp_isis_civ_events_adm3prec']
ucdp_conflict_precadm3.loc[:,'ucdp_all_events_adm3prec'] = ucdp_conflict_precadm3['ucdp_nonisis_civ_events_adm3prec'] + ucdp_conflict_precadm3['ucdp_isis_civ_events_adm3prec'] + ucdp_conflict_precadm3['ucdp_isis_nonciv_events_adm3prec'] + ucdp_conflict_precadm3['ucdp_nonisis_nonciv_events_adm3prec']

ucdp_conflict_precadm3.head()

(11997, 6)
(3567, 7)


/tmp/ipykernel_2200/1551171189.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ucdp_conflict_precadm3.loc[:,'isis_civ'] = ucdp_conflict_precadm3['isis_conflict'].astype(str) + "_" + ucdp_conflict_precadm3['civ_violence'].astype(str)


isis_civ,ADM3_EN,month,ucdp_nonisis_nonciv_events_adm3prec,ucdp_nonisis_civ_events_adm3prec,ucdp_isis_nonciv_events_adm3prec,ucdp_isis_civ_events_adm3prec,ucdp_all_isis_events_adm3prec,ucdp_all_civ_events_adm3prec,ucdp_all_events_adm3prec
0,Abu Dalf,2008-02-29,0.0,0.0,1.0,0.0,1.0,0.0,1.0
1,Abu Dalf,2008-09-30,0.0,0.0,1.0,0.0,1.0,0.0,1.0
2,Abu Dalf,2011-02-28,0.0,0.0,1.0,0.0,1.0,0.0,1.0
3,Abu Dalf,2013-06-30,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,Abu Dalf,2015-01-31,0.0,0.0,1.0,0.0,1.0,0.0,1.0


In [96]:
# sum events at the admin 2 level of precision or better (where_prec == 1, 2, or 3)
print(ucdp_conflict.shape)
ucdp_conflict_precadm2 = ucdp_conflict[ucdp_conflict['where_prec'].isin([1,2,3])]
ucdp_conflict_precadm2.loc[:,'isis_civ'] = ucdp_conflict_precadm2['isis_conflict'].astype(str) + "_" + ucdp_conflict_precadm2['civ_violence'].astype(str)
print(ucdp_conflict_precadm2.shape)
ucdp_conflict_precadm2 = ucdp_conflict_precadm2.groupby(['ADM3_EN','isis_civ','month'])['num_events'].sum().reset_index()
ucdp_conflict_precadm2 = ucdp_conflict_precadm2.pivot(index=['ADM3_EN','month'],columns='isis_civ',values='num_events').reset_index()
ucdp_conflict_precadm2 = ucdp_conflict_precadm2.rename(columns={"0_0": 'ucdp_nonisis_nonciv_events_adm2prec',
                                                                "0_1": 'ucdp_nonisis_civ_events_adm2prec',
                                                                "1_0": 'ucdp_isis_nonciv_events_adm2prec',
                                                                "1_1": 'ucdp_isis_civ_events_adm2prec'})
ucdp_conflict_precadm2 = ucdp_conflict_precadm2.fillna(value={'ucdp_nonisis_nonciv_events_adm2prec':0,
                                                               'ucdp_nonisis_civ_events_adm2prec':0,
                                                               'ucdp_isis_nonciv_events_adm2prec':0,
                                                               'ucdp_isis_civ_events_adm2prec':0})
ucdp_conflict_precadm2.loc[:,'ucdp_all_isis_events_adm2prec'] = ucdp_conflict_precadm2['ucdp_isis_nonciv_events_adm2prec'] + ucdp_conflict_precadm2['ucdp_isis_civ_events_adm2prec']
ucdp_conflict_precadm2.loc[:,'ucdp_all_civ_events_adm2prec'] = ucdp_conflict_precadm2['ucdp_nonisis_civ_events_adm2prec'] + ucdp_conflict_precadm2['ucdp_isis_civ_events_adm2prec']
ucdp_conflict_precadm2.loc[:,'ucdp_all_events_adm2prec'] = ucdp_conflict_precadm2['ucdp_nonisis_civ_events_adm2prec'] + ucdp_conflict_precadm2['ucdp_isis_civ_events_adm2prec'] + ucdp_conflict_precadm2['ucdp_isis_nonciv_events_adm2prec'] + ucdp_conflict_precadm2['ucdp_nonisis_nonciv_events_adm2prec']
ucdp_conflict_precadm2.head()

(11997, 6)
(4387, 7)


/tmp/ipykernel_2200/1009229424.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ucdp_conflict_precadm2.loc[:,'isis_civ'] = ucdp_conflict_precadm2['isis_conflict'].astype(str) + "_" + ucdp_conflict_precadm2['civ_violence'].astype(str)


isis_civ,ADM3_EN,month,ucdp_nonisis_nonciv_events_adm2prec,ucdp_nonisis_civ_events_adm2prec,ucdp_isis_nonciv_events_adm2prec,ucdp_isis_civ_events_adm2prec,ucdp_all_isis_events_adm2prec,ucdp_all_civ_events_adm2prec,ucdp_all_events_adm2prec
0,Abu Dalf,2008-02-29,0.0,0.0,1.0,0.0,1.0,0.0,1.0
1,Abu Dalf,2008-09-30,0.0,0.0,1.0,0.0,1.0,0.0,1.0
2,Abu Dalf,2011-02-28,0.0,0.0,1.0,0.0,1.0,0.0,1.0
3,Abu Dalf,2013-06-30,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,Abu Dalf,2014-06-30,0.0,0.0,1.0,0.0,1.0,0.0,1.0


In [97]:
print(ucdp_conflict.shape)
ucdp_conflict_precadm1 = ucdp_conflict[ucdp_conflict['where_prec'].isin([1,2,3,4])]
ucdp_conflict_precadm1.loc[:,'isis_civ'] = ucdp_conflict_precadm1['isis_conflict'].astype(str) + "_" + ucdp_conflict_precadm1['civ_violence'].astype(str)
print(ucdp_conflict_precadm1.shape)
ucdp_conflict_precadm1 = ucdp_conflict_precadm1.groupby(['ADM3_EN','isis_civ','month'])['num_events'].sum().reset_index()
ucdp_conflict_precadm1 = ucdp_conflict_precadm1.pivot(index=['ADM3_EN','month'],columns='isis_civ',values='num_events').reset_index()
ucdp_conflict_precadm1 = ucdp_conflict_precadm1.rename(columns={"0_0": 'ucdp_nonisis_nonciv_events_adm1prec',
                                                                "0_1": 'ucdp_nonisis_civ_events_adm1prec',
                                                                "1_0": 'ucdp_isis_nonciv_events_adm1prec',
                                                                "1_1": 'ucdp_isis_civ_events_adm1prec'})
ucdp_conflict_precadm1 = ucdp_conflict_precadm1.fillna(value={'ucdp_nonisis_nonciv_events_adm1prec':0,
                                                               'ucdp_nonisis_civ_events_adm1prec':0,
                                                               'ucdp_isis_nonciv_events_adm1prec':0,
                                                               'ucdp_isis_civ_events_adm1prec':0})
ucdp_conflict_precadm1.loc[:,'ucdp_all_isis_events_adm1prec'] = ucdp_conflict_precadm1['ucdp_isis_nonciv_events_adm1prec'] + ucdp_conflict_precadm1['ucdp_isis_civ_events_adm1prec']
ucdp_conflict_precadm1.loc[:,'ucdp_all_civ_events_adm1prec'] = ucdp_conflict_precadm1['ucdp_nonisis_civ_events_adm1prec'] + ucdp_conflict_precadm1['ucdp_isis_civ_events_adm1prec']
ucdp_conflict_precadm1.loc[:,'ucdp_all_events_adm1prec'] = ucdp_conflict_precadm1['ucdp_nonisis_civ_events_adm1prec'] + ucdp_conflict_precadm1['ucdp_isis_civ_events_adm1prec'] + ucdp_conflict_precadm1['ucdp_isis_nonciv_events_adm1prec'] + ucdp_conflict_precadm1['ucdp_nonisis_nonciv_events_adm1prec']
print(ucdp_conflict_precadm1.shape)

ucdp_conflict_precadm1.head()

(11997, 6)
(11997, 7)
(9696, 9)


isis_civ,ADM3_EN,month,ucdp_nonisis_nonciv_events_adm1prec,ucdp_nonisis_civ_events_adm1prec,ucdp_isis_nonciv_events_adm1prec,ucdp_isis_civ_events_adm1prec,ucdp_all_isis_events_adm1prec,ucdp_all_civ_events_adm1prec,ucdp_all_events_adm1prec
0,Abi Gharaq,2013-06-30,0.0,0.0,1.0,1.0,2.0,1.0,2.0
1,Abi Gharaq,2013-11-30,0.0,0.0,1.0,0.0,1.0,0.0,1.0
2,Abi Gharaq,2014-06-30,0.0,0.0,1.0,0.0,1.0,0.0,1.0
3,Abi Gharaq,2014-07-31,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,Abu Dalf,2006-05-31,0.0,0.0,1.0,0.0,1.0,0.0,1.0


In [98]:
ucdp_conflict_full = pd.merge(ucdp_conflict_precadm1,ucdp_conflict_precadm2,on=['ADM3_EN','month'],
                              how='outer')
ucdp_conflict_full = pd.merge(ucdp_conflict_full,ucdp_conflict_precadm3,on=['ADM3_EN','month'],
                              how='outer')
ucdp_conflict_full = ucdp_conflict_full.fillna(0)
print(ucdp_conflict_full.shape)

(9696, 23)


In [99]:
# test that logic holds, no more adm3-precision events than adm 2, than admin 1
ucdp_conflict_full[ucdp_conflict_full['ucdp_all_events_adm2prec'] < ucdp_conflict_full['ucdp_all_events_adm3prec'] ]

isis_civ,ADM3_EN,month,ucdp_nonisis_nonciv_events_adm1prec,ucdp_nonisis_civ_events_adm1prec,ucdp_isis_nonciv_events_adm1prec,ucdp_isis_civ_events_adm1prec,ucdp_all_isis_events_adm1prec,ucdp_all_civ_events_adm1prec,ucdp_all_events_adm1prec,ucdp_nonisis_nonciv_events_adm2prec,...,ucdp_all_isis_events_adm2prec,ucdp_all_civ_events_adm2prec,ucdp_all_events_adm2prec,ucdp_nonisis_nonciv_events_adm3prec,ucdp_nonisis_civ_events_adm3prec,ucdp_isis_nonciv_events_adm3prec,ucdp_isis_civ_events_adm3prec,ucdp_all_isis_events_adm3prec,ucdp_all_civ_events_adm3prec,ucdp_all_events_adm3prec


In [101]:
# test that logic holds, fewer isis events less than overall events
ucdp_conflict_full[ucdp_conflict_full['ucdp_all_isis_events_adm3prec'] > ucdp_conflict_full['ucdp_all_events_adm3prec'] ]

isis_civ,ADM3_EN,month,ucdp_nonisis_nonciv_events_adm1prec,ucdp_nonisis_civ_events_adm1prec,ucdp_isis_nonciv_events_adm1prec,ucdp_isis_civ_events_adm1prec,ucdp_all_isis_events_adm1prec,ucdp_all_civ_events_adm1prec,ucdp_all_events_adm1prec,ucdp_nonisis_nonciv_events_adm2prec,...,ucdp_all_isis_events_adm2prec,ucdp_all_civ_events_adm2prec,ucdp_all_events_adm2prec,ucdp_nonisis_nonciv_events_adm3prec,ucdp_nonisis_civ_events_adm3prec,ucdp_isis_nonciv_events_adm3prec,ucdp_isis_civ_events_adm3prec,ucdp_all_isis_events_adm3prec,ucdp_all_civ_events_adm3prec,ucdp_all_events_adm3prec


In [102]:
# store UCDP conflict data at month-adm3 level
ucdp_conflict_full_file = source_dir + '/data/conflict/ucdp_conflict_month_adm3_level.csv'

ucdp_conflict_full.to_csv(ucdp_conflict_full_file,index=False)

In [103]:
# add in UCDP conflict data
print(adm3_iraq_data.shape)
ucdp_conflict_full['month'] = pd.to_datetime(ucdp_conflict_full['month'])
adm3_iraq_data = adm3_iraq_data.merge(ucdp_conflict_full,
                                            left_on=['ADM3_EN','date'],
                                           right_on=['ADM3_EN','month'],
                                           how='left')
adm3_iraq_data = adm3_iraq_data.drop(columns=['month','adm3_en'])
print(adm3_iraq_data.shape)

(84966, 36)
(84966, 56)


In [104]:
# attach Adm3 geometries and parent areas to final dataset
iraq_shp_adm3_filt = iraq_shp_adm3.loc[:,('Shape_Area', 'ADM3_EN', 'ADM3_AR', 'ADM3_PCODE',
       'ADM2_EN', 'ADM2_AR', 'ADM2_PCODE', 'ADM1_EN', 'ADM1_AR', 'ADM1_PCODE',
       'geometry')]
print(adm3_iraq_data.shape)
adm3_iraq_data_fin = adm3_iraq_data.merge(iraq_shp_adm3_filt, how='left',
                                          on='ADM3_EN')
adm3_iraq_data_fin = adm3_iraq_data_fin.rename(columns={'date':'month'})
print(adm3_iraq_data_fin.shape)

(84966, 56)
(84966, 66)


In [105]:
adm3_iraq_data_fin[adm3_iraq_data_fin.duplicated(subset=['month','ADM3_EN'],keep=False)]

,ADM3_EN,month,mean_evi_nomask,mean_evi_nomask_scaled,max_evi_nomask,max_evi_nomask_scaled,ACTION_IND,ACTION_DIR,ACTION_PRT,mean_evi_mask,...,Shape_Area,ADM3_AR,ADM3_PCODE,ADM2_EN,ADM2_AR,ADM2_PCODE,ADM1_EN,ADM1_AR,ADM1_PCODE,geometry


In [106]:
adm3_iraq_data_fin.columns

Index(['ADM3_EN', 'month', 'mean_evi_nomask', 'mean_evi_nomask_scaled',
       'max_evi_nomask', 'max_evi_nomask_scaled', 'ACTION_IND', 'ACTION_DIR',
       'ACTION_PRT', 'mean_evi_mask', 'mean_evi_mask_scaled', 'max_evi_mask',
       'max_evi_mask_scaled', 'mean_ndvi', 'mean_ndvi_scaled', 'max_ndvi',
       'max_ndvi_scaled', 'pop_count', 'pop_density', 'iom_attacked',
       'iom_occupied', 'iom_isil_ingroup_prior', 'iom_isil_ingroup_curr',
       'iom_disputed_area', 'iom_retaken_year', 'iom_retaken_post_2016',
       'iom_no_isil_action', 'perc_isis_control', 'date_month_end',
       'sunni_dom', 'sunni_mix', 'no_sunni', 'iom_occ_start_month',
       'iom_occ_end_month', 'iom_occ_length_mon',
       'ucdp_nonisis_nonciv_events_adm1prec',
       'ucdp_nonisis_civ_events_adm1prec', 'ucdp_isis_nonciv_events_adm1prec',
       'ucdp_isis_civ_events_adm1prec', 'ucdp_all_isis_events_adm1prec',
       'ucdp_all_civ_events_adm1prec', 'ucdp_all_events_adm1prec',
       'ucdp_nonisis_nonciv_e

In [107]:
monthly_adm3_geo_file = source_dir + '/data/combined/monthly_adm3_data.geojson'
adm3_iraq_data_fin_gpd = gpd.GeoDataFrame(adm3_iraq_data_fin,geometry=adm3_iraq_data_fin['geometry'])
adm3_iraq_data_fin_gpd.to_file(monthly_adm3_geo_file,driver='GeoJSON')

In [108]:
monthly_adm3_file = source_dir + '/data/combined/monthly_adm3_data.csv'
adm3_iraq_data_fin.drop(columns=['geometry']).to_csv(monthly_adm3_file,index=False,
                                                     encoding='utf-8-sig')